## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import challenge_g_key

# Configure gmaps API key
gmaps.configure(api_key=challenge_g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Sao Joao Da Barra,BR,-21.64,-41.05,74.16,88,37,12.03,light rain
1,1,Havre-Saint-Pierre,CA,50.23,-63.60,28.40,92,75,4.70,broken clouds
2,2,Muros,ES,42.78,-9.06,53.01,90,70,1.01,light rain
3,3,Atuona,PF,-9.80,-139.03,79.34,72,1,16.02,clear sky
4,4,Bluff,NZ,-46.60,168.33,60.01,65,6,10.00,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
max_temp = float(input("What is the maximum temperature you would like?"))
min_temp = float(input("What is the mininum temperature you would like?"))

What is the maximum temperature you would like?70
What is the mininum temperature you would like?55


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
vacation_data_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
vacation_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Bluff,NZ,-46.60,168.33,60.01,65,6,10.00,clear sky
5,5,Castro,BR,-24.79,-50.01,63.59,96,95,5.95,heavy intensity rain
6,6,Bilma,NE,18.69,12.92,66.58,27,0,5.64,clear sky
14,14,Cascais,PT,38.70,-9.42,57.99,77,30,14.18,light rain
15,15,Nueve De Julio,AR,-35.44,-60.88,66.99,96,0,1.99,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
vacation_data_df.isnull().sum()

City_ID                0
City                   0
Country                4
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
vacation_data_updated_df = vacation_data_df.dropna()
vacation_data_updated_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = vacation_data_updated_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Bluff,NZ,60.01,clear sky,-46.60,168.33,
5,Castro,BR,63.59,heavy intensity rain,-24.79,-50.01,
6,Bilma,NE,66.58,clear sky,18.69,12.92,
14,Cascais,PT,57.99,light rain,38.70,-9.42,
15,Nueve De Julio,AR,66.99,clear sky,-35.44,-60.88,
16,Kruisfontein,ZA,59.00,clear sky,-34.00,24.73,
20,Mataura,NZ,60.01,broken clouds,-46.19,168.86,
22,Ginir,ET,58.03,broken clouds,7.13,40.70,
23,Kidal,ML,65.80,clear sky,18.44,1.41,
29,Uthal,PK,66.13,clear sky,25.81,66.62,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": challenge_g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df.head()

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Bluff,NZ,60.01,clear sky,-46.60,168.33,Bluff Homestead - Guesthouse & Campervan Park
5,Castro,BR,63.59,heavy intensity rain,-24.79,-50.01,CHACARA BAILLY
6,Bilma,NE,66.58,clear sky,18.69,12.92,Expeditions Ténére voyages
14,Cascais,PT,57.99,light rain,38.70,-9.42,Hotel Vila Galé Cascais
15,Nueve De Julio,AR,66.99,clear sky,-35.44,-60.88,Grand Hotel Libertad - GHL


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} and {Max Temp}°F</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
city = hotel_df["City"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))